In [2]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import kagglehub
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from kagglehub import KaggleDatasetAdapter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import layers, models, regularizers


# Set the path to the file you'd like to load
file_path = "youtoxic_english_1000.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "reihanenamdari/youtube-toxicity-data",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

2025-11-28 17:25:39.317545: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/cn/0dflk63d5lj2brskz94j7c880000gn/T/ipykernel_39470/2890072755.py:24: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


In [3]:
df.head()

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


In [4]:
# Keep only the relevant columns
df = df[['Text', 'IsToxic']]

In [5]:
# Convert 'IsToxic' to integer type
df['IsToxic'] = df['IsToxic'].astype(int)

In [6]:
# cleaning for deep learning
def clean_text(t):
    t = str(t).lower()
    t = re.sub(r"http\S+", "", t)                # remove URLs
    t = re.sub(r"@\w+", "", t)                   # remove @username
    t = re.sub(r"[^a-z0-9\s!?']", " ", t)        # keep letters, digits, !, ?, '
    t = re.sub(r"\s+", " ", t)                   # collapse multiple spaces
    return t.strip()
df['Text'] = df['Text'].apply(clean_text)

In [7]:
df.head()

,Text,IsToxic
0,if only people would just take a step back and...,0
1,law enforcement is not trained to shoot to app...,1
2,dont you reckon them 'black lives matter' bann...,1
3,there are a very large number of people who do...,0
4,the arab dude is absolutely right he should ha...,0


In [8]:
X = df['Text'].values
y = df['IsToxic'].values.astype(int)

# 70% train, 15% val, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

len(X_train), len(X_val), len(X_test)

(700, 150, 150)

In [9]:
MAX_VOCAB_SIZE = 5000
MAX_LEN = 80

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

# Convert text → sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

# Padding
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

print("Tokenizing…")
X_train_pad.shape, X_val_pad.shape, X_test_pad.shape

Tokenizing…


((700, 80), (150, 80), (150, 80))

In [ ]:
EMBEDDING_DIM = 64
LSTM_UNITS = 32

vocab_size = min(MAX_VOCAB_SIZE, len(tokenizer.word_index) + 1)

model = models.Sequential([
    layers.Embedding(
        input_dim=vocab_size,
        output_dim=EMBEDDING_DIM,
        input_length=MAX_LEN
    ),
    layers.Bidirectional(layers.LSTM(LSTM_UNITS)),
    # To prevent overfitting
    layers.Dropout(0.3),
    # toxic/non-toxic non-binary classification
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    # loss function for binary classification
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

/Users/lee/anaconda3/envs/env_3/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)

history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.5329 - loss: 0.6913 - val_accuracy: 0.5333 - val_loss: 0.6904
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.5500 - loss: 0.6795 - val_accuracy: 0.5333 - val_loss: 0.6838
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.6571 - loss: 0.6368 - val_accuracy: 0.5800 - val_loss: 0.6568
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.8143 - loss: 0.4954 - val_accuracy: 0.6533 - val_loss: 0.6493
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.8957 - loss: 0.3343 - val_accuracy: 0.6533 - val_loss: 0.6909
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9614 - loss: 0.1924 - val_accuracy: 0.6467 - val_loss: 0.7323
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9829 - loss: 0.1116 - val_accuracy: 0.5600 - val_loss: 0.8406
Epoch 8/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9900 - loss: 0.0851 - val_accuracy: 0.6533 - v

In [12]:
# 1. Final training accuracy
train_loss, train_acc = model.evaluate(X_train_pad, y_train, verbose=0)

print(f"Final Train Loss: {train_loss:.4f}")
print(f"Final Train Accuracy: {train_acc:.4f}")

# 2. Final test accuracy
test_loss, test_acc = model.evaluate(X_test_pad, y_test, verbose=0)

print(f"Final Test Loss: {test_loss:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

# 3. Predictions
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob >= 0.5).astype(int)

# 4. Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))

# 5. Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)


Final Train Loss: 0.3672
Final Train Accuracy: 0.9229
Final Test Loss: 0.6248
Final Test Accuracy: 0.6000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step

Classification Report:
              precision    recall  f1-score   support

           0     0.6296    0.6296    0.6296        81
           1     0.5652    0.5652    0.5652        69

    accuracy                         0.6000       150
   macro avg     0.5974    0.5974    0.5974       150
weighted avg     0.6000    0.6000    0.6000       150


Confusion Matrix:
[[51 30]
 [30 39]]
